In [1]:
import cv2
import os
import tensorflow as tf
import numpy as np

import cv2
import os

def extract_frames(video_path, output_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps

    # Calculate the frame interval to get n/2 frames
    frame_interval = int((total_frames * 4) /  duration)

    # Create output directory if it doesn't exist
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # Loop through the video and extract frames
    current_frame = 0
    while True:
        ret, frame = cap.read()
        
        if not ret:
            break  # Break the loop if no more frames are available

        # Save the frame if it's at the interval
        if current_frame % frame_interval == 0:
            frame_filename = f"{output_path}/frame_{current_frame}.jpg"
            cv2.imwrite(frame_filename, frame)

        current_frame += 1

    # Release the video capture object
    cap.release()




In [2]:
def predict_emotion(frame_path):
    img = cv2.imread(frame_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale if needed
    img = cv2.resize(img, (48, 48))  # Resize to match the input size of the model
    img = img / 255.0  # Normalize pixel values
    
    img = img.reshape((1,) + img.shape + (1,))
    
    # Make prediction
    emotion_probs = emotion_model.predict(img)
    
    # Get the emotion with the highest probability
    predicted_emotion = emotion_classes[np.argmax(emotion_probs)]
    
    return predicted_emotion


In [3]:
def calculate_emotion_score(emotion):
    emotion_scores = {'angry': 40, 'fear': 40, 'happy': 100, 'neutral': 65, 'sad': 50, 'surprise': 55}
    
    return emotion_scores.get(emotion, 0)



In [7]:
# Load the pre-trained emotion detection model
model_path = r"D:\AI_interview\facerec\emotion_detector_final_model.h5"
emotion_model = tf.keras.models.load_model(model_path)

# Define the emotion classes
emotion_classes = ['angry', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Specify video path and output directory
video_path = r"D:\AI_interview\recorded\answer_4.avi"
output_path = r"D:\AI_interview\extracted"

# Extract frames from the video
extract_frames(video_path, output_path)

# Check the content of the output directory
print("Frames extracted:")
print(os.listdir(output_path))

# Predict emotions for each frame and calculate average emotion score
emotion_scores = []

for frame_file in os.listdir(output_path):
    frame_path = os.path.join(output_path, frame_file)
    predicted_emotion = predict_emotion(frame_path)
    emotion_score = calculate_emotion_score(predicted_emotion)
    emotion_scores.append(emotion_score)

# Check the content of emotion_scores
print("Emotion Scores:")
print(emotion_scores)

# Check if emotion_scores is empty
if not emotion_scores:
    print("No frames were processed. Please check the frame extraction process.")
else:
    # Calculate average emotion score
    average_emotion_score = np.nanmean(emotion_scores)
    print("Average Emotion Score:", average_emotion_score)

Frames extracted:
['frame_0.jpg', 'frame_1040.jpg', 'frame_1120.jpg', 'frame_1200.jpg', 'frame_1280.jpg', 'frame_1360.jpg', 'frame_1440.jpg', 'frame_160.jpg', 'frame_240.jpg', 'frame_320.jpg', 'frame_400.jpg', 'frame_480.jpg', 'frame_560.jpg', 'frame_640.jpg', 'frame_720.jpg', 'frame_80.jpg', 'frame_800.jpg', 'frame_880.jpg', 'frame_960.jpg']
1/1 [==============================] - 0s 33ms/step
Emotion Scores:
[65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65]
Average Emotion Score: 65.0
